In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

#text processing
import nltk #nltk oder spacey
import string
import re #regular expression

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


#model(hugging face)
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_scheduler

2025-01-22 16:54:28.671155: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 16:54:29.331632: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-22 16:54:29.331676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-22 16:54:29.331708: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 16:54:29.360803: I tensorflow/core/platform/cpu_feature_g

In [3]:
df = pd.read_excel('df_complete_cleansing.xlsx', index_col=0)
df

,ReceiverID,ActionType,NegoOutcome,Content,Content_Length,Sentence_Count,Word_Count_nltk,NegoOutcomeLabel,Sentiment,SentimentCategory
SenderID,,,,,,,,,,
70,69,Offer,FinalAccept,hope well management company agreed building f...,2529,17,470,1,0.213699,Positive
69,70,Counteroffer,FinalAccept,reaching mei apologize slight delay getting ba...,2579,21,483,1,0.165002,Positive
70,69,Counteroffer,FinalAccept,r kind response need hurry guess face daily fi...,2336,15,454,1,0.222533,Positive
69,70,Counteroffer,FinalAccept,swift reply read set proposal discussed collea...,1961,13,381,1,0.160333,Positive
70,69,Counteroffer,FinalAccept,proposal effort far seems getting close resolv...,1917,10,376,1,0.122500,Positive
...,...,...,...,...,...,...,...,...,...,...
851,856,Counteroffer,FinalReject,latest offer point completely let u take corpo...,1755,21,358,0,0.285340,Positive
856,851,Counteroffer,FinalReject,much latest offer happy could already reached ...,1460,17,294,0,0.282807,Positive
851,856,Counteroffer,FinalReject,meyer fast answer happy found solution issue n...,673,10,144,0,0.251786,Positive


In [4]:
df['Content'] = df['Content'].astype(str)

In [5]:
nego_message_data=df['Content']
df_nego_message_data = pd.DataFrame(nego_message_data)
df_nego_message_data

,Content
SenderID,
70,hope well management company agreed building f...
69,reaching mei apologize slight delay getting ba...
70,r kind response need hurry guess face daily fi...
69,swift reply read set proposal discussed collea...
70,proposal effort far seems getting close resolv...
...,...
851,latest offer point completely let u take corpo...
856,much latest offer happy could already reached ...
851,meyer fast answer happy found solution issue n...


In [6]:
df_nego_message_data['Content']

SenderID
70     hope well management company agreed building f...
69     reaching mei apologize slight delay getting ba...
70     r kind response need hurry guess face daily fi...
69     swift reply read set proposal discussed collea...
70     proposal effort far seems getting close resolv...
                             ...                        
851    latest offer point completely let u take corpo...
856    much latest offer happy could already reached ...
851    meyer fast answer happy found solution issue n...
856    still interested coming joint result neverthel...
851    sorry accept less understand argument like las...
Name: Content, Length: 2332, dtype: object

In [7]:
sentences = df_nego_message_data.Content.values
sentences = ["[CLS]"+ sentence +" [SEP]" for sentence in sentences]

labels= df.NegoOutcomeLabel.values

In [8]:
sentences

['[CLS]hope well management company agreed building fruitful partnership face upcoming disruption market best innovation green spirit would like introduce nasa nova reaching tighten general term harm ag established round discussion far please find attached proposal open issue joint venture would like explain following director would like address unsolved issue number director sent since agreed decisionmaking committee consisting member simple majority power strongly suggest dividing seat svn tech would able deal local authority partner successfully using strong local network relationship engineer develop thorough convenient product sharing experience side inspiring essential thus looking forward set production process valuable input engineer work together make adaption improve product experience may take common worker establishing high joint utility seek solution equally distributed asset facing u continue paying worker would appreciate harm ag could take care potentially hired common 

In [9]:
labels

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])
print(tokenized_texts[1])

['[CLS]', 'hope', 'well', 'management', 'company', 'agreed', 'building', 'fruit', '##ful', 'partnership', 'face', 'upcoming', 'disruption', 'market', 'best', 'innovation', 'green', 'spirit', 'would', 'like', 'introduce', 'nasa', 'nova', 'reaching', 'tighten', 'general', 'term', 'harm', 'ag', 'established', 'round', 'discussion', 'far', 'please', 'find', 'attached', 'proposal', 'open', 'issue', 'joint', 'venture', 'would', 'like', 'explain', 'following', 'director', 'would', 'like', 'address', 'un', '##sol', '##ved', 'issue', 'number', 'director', 'sent', 'since', 'agreed', 'decision', '##making', 'committee', 'consisting', 'member', 'simple', 'majority', 'power', 'strongly', 'suggest', 'dividing', 'seat', 'sv', '##n', 'tech', 'would', 'able', 'deal', 'local', 'authority', 'partner', 'successfully', 'using', 'strong', 'local', 'network', 'relationship', 'engineer', 'develop', 'thorough', 'convenient', 'product', 'sharing', 'experience', 'side', 'inspiring', 'essential', 'thus', 'looking

In [11]:
MAX_LEN = 512

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [12]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

In [13]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [14]:
train_inputs, validation_inputs, train_labels, validation_labels= train_test_split(input_ids, labels, random_state=42, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.3)

In [15]:
train_inputs

array([[  101,  2720, 11527, ...,     0,     0,     0],
       [  101,  2171,  3076, ...,     0,     0,     0],
       [  101,  4541,  3025, ...,     0,     0,     0],
       ...,
       [  101,  6854,  2172, ...,     0,     0,     0],
       [  101,  3100,  3305, ...,     0,     0,     0],
       [  101,  2215, 12367, ...,     0,     0,     0]])

In [16]:
validation_inputs

array([[  101,  2720, 27916, ...,     0,     0,     0],
       [  101,  2720,  7537, ...,     0,     0,     0],
       [  101,  4248,  3433, ...,     0,     0,     0],
       ...,
       [  101, 13676,  3988, ...,     0,     0,     0],
       [  101, 27916,  2116, ...,     0,     0,     0],
       [  101,  2748,  2228, ...,     0,     0,     0]])

In [17]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [18]:
#je mehr desto langsamer -> batchsize immer wieder ein bisschen anpassen und ausprobieren
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader =DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [19]:
# Device Setup: Force GPU if available
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Force CPU usage
device = torch.device("cuda")
#print("Using CPU for all computations.")

In [20]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2) 
model.to(device)
model.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [21]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [22]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):
            ### Prepare data
            input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
            outputs = model(input_ids, attention_mask=input_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']
        
            _, predicted_labels = torch.max(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()

    return correct_pred.float()/num_examples * 100

In [23]:
import time

start_time = time.time()

NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_dataloader):
        ### Prepare data
        input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
        ### Forward
        outputs = model(input_ids, attention_mask=input_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
    
        ### Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_dataloader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, validation_dataloader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 0001/0005 | Batch 0000/0051 | Loss: 0.6930
training accuracy: 68.50%
valid accuracy: 69.43%
Time elapsed: 0.72 min
Epoch: 0002/0005 | Batch 0000/0051 | Loss: 0.5632
training accuracy: 68.57%
valid accuracy: 69.57%
Time elapsed: 1.27 min
Epoch: 0003/0005 | Batch 0000/0051 | Loss: 0.6883
training accuracy: 74.08%
valid accuracy: 72.14%
Time elapsed: 1.82 min
Epoch: 0004/0005 | Batch 0000/0051 | Loss: 0.4128
training accuracy: 84.99%
valid accuracy: 74.29%
Time elapsed: 2.37 min
Epoch: 0005/0005 | Batch 0000/0051 | Loss: 0.3594
training accuracy: 75.25%
valid accuracy: 71.71%
Time elapsed: 2.93 min
Total Training Time: 2.93 min


In [24]:
import time

start_time = time.time()

NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_dataloader):
        ### Prepare data
        input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
        ### Forward
        outputs = model(input_ids, attention_mask=input_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
    
        ### Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_dataloader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, validation_dataloader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 0001/0010 | Batch 0000/0051 | Loss: 0.4512
training accuracy: 97.06%
valid accuracy: 74.71%
Time elapsed: 0.56 min
Epoch: 0002/0010 | Batch 0000/0051 | Loss: 0.1712
training accuracy: 99.39%
valid accuracy: 75.29%
Time elapsed: 1.11 min
Epoch: 0003/0010 | Batch 0000/0051 | Loss: 0.0426
training accuracy: 96.38%
valid accuracy: 75.14%
Time elapsed: 1.67 min
Epoch: 0004/0010 | Batch 0000/0051 | Loss: 0.0545
training accuracy: 97.43%
valid accuracy: 74.86%
Time elapsed: 2.22 min
Epoch: 0005/0010 | Batch 0000/0051 | Loss: 0.1136
training accuracy: 99.69%
valid accuracy: 75.29%
Time elapsed: 2.78 min
Epoch: 0006/0010 | Batch 0000/0051 | Loss: 0.0060
training accuracy: 99.94%
valid accuracy: 76.29%
Time elapsed: 3.33 min
Epoch: 0007/0010 | Batch 0000/0051 | Loss: 0.0021
training accuracy: 99.94%
valid accuracy: 76.71%
Time elapsed: 3.89 min
Epoch: 0008/0010 | Batch 0000/0051 | Loss: 0.0024
training accuracy: 99.39%
valid accuracy: 73.43%
Time elapsed: 4.45 min
Epoch: 0009/0010 | Batch